In [6]:
import pandas as pd
import itertools
import numpy as np
from IPython.display import display, Markdown, Latex
import json

DATASETS = {
    "fb.csv": "fb_200M_uint64",
    "wiki.csv": "wiki_ts_200M_uint64",
    "osm.csv": "osm_cellids_200M_uint64",
    "osm400.csv": "osm_cellids_400M_uint64",
    "osm600.csv": "osm_cellids_600M_uint64",
    "osm800.csv": "osm_cellids_800M_uint64",
    "books.csv": "books_200M_uint64",
    "books400.csv": "books_400M_uint64",
    "books600.csv": "books_600M_uint64",
    "books800.csv": "books_800M_uint64"   
}

In [17]:

def produce_front(opt_out_fn, debug=False):    
    df = pd.read_csv(opt_out_fn)
    to_search = df[(df["average log2 error"] < 15) & (df["size binary search"] < 2000000000)].sort_values("average log2 error").copy()
    if debug:
        display(Markdown(f"# Top 20 overall `{opt_out_fn}`"))
        display(to_search.sort_values("average log2 error").iloc[0:20])
    
    front = to_search[to_search.front & (to_search["average log2 error"] < 15) & (to_search["size binary search"] < 2000000000)].copy()
    if debug:
        display(Markdown(f"# Pareto front `{opt_out_fn}`"))
        display(front.sort_values("average log2 error"))
    front = front.to_dict("record")
    
    retained = []
    for rmi1 in front:
        for rmi2 in front:
            if rmi1["layers"] == rmi2["layers"] and rmi1["branching factor"] == rmi2["branching factor"]:
                continue

            if abs(rmi1["average log2 error"] - rmi2["average log2 error"]) < 0.001:
                if rmi1["size binary search"] > rmi2["size binary search"]:
                    break

            if abs(rmi1["size binary search"] - rmi2["size binary search"]) < 1000:
                if rmi1["average log2 error"] > rmi2["average log2 error"]:
                    break
        else:
            retained.append(rmi1)

    retained = pd.DataFrame(retained).sort_values("average log2 error")
    if debug:
        display(Markdown(f"# Pruned `{opt_out_fn}`"))
        display(retained)

    retained["diff"] = [-1_000_000_000_000] + list(np.diff(retained.sort_values("size binary search", ascending=False)["size binary search"].to_numpy()))
    selected = retained.sort_values("diff", ascending=True)[0:10]
    selected = selected.sort_values("size binary search", ascending=False)
    display(Markdown(f"# Final configs `{opt_out_fn}`"))
    
    tmp = selected.copy()
    while len(tmp) < 10:
        tmp = tmp.append(tmp.iloc[-1], ignore_index=True)
    tmp = tmp.reset_index().drop(columns="index").reset_index()
    display(tmp)

    tmp["namespace"] = tmp.apply(lambda x: DATASETS[opt_out_fn] + "_" + str(x["index"]), axis=1)
    spec = {"configs" : tmp[["layers", "branching factor", "namespace", "binary"]].to_dict("record")}
    
        
    return spec

In [18]:
for k, v in DATASETS.items():
    front = produce_front(k)
    assert len(front["configs"]) == 10
    with open(f"/home/ryan/mcgraw/SOSD-private/scripts/rmi_specs/{v}.json", "w") as f:
        json.dump(front, f)

# Final configs `fb.csv`

,index,layers,branching factor,average error,average error %,average l2 error,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front,diff
0,0,"linear_spline,linear",2097152,127.140537,0.001914,4.226717e+07,7.016945,3828,0.001914,50331664,33554448,nm77680171b4d11b70aa2f,True,False,True,-1000000000000
1,1,"linear_spline,linear",1048576,183.943797,0.002550,9.123860e+07,7.723291,5099,0.002550,25165840,16777232,nm9d7314ecfb0606fea357,True,False,True,-25165824
2,2,"linear_spline,linear",524288,266.117007,0.002589,2.087363e+08,8.428844,5178,0.002589,12582928,8388624,nm14697a215f79906b7b7d,True,False,True,-12582912
3,3,"linear_spline,linear",262144,383.549167,0.002908,5.037689e+08,9.120617,5816,0.002908,6291472,4194320,nmd985905d60efa476204b,True,False,True,-6291456
4,4,"linear,linear",131072,554.474509,0.005482,1.338033e+09,9.793821,10964,0.005482,3145744,2097168,nm4ba04214fe8ae1c6a0ef,True,False,True,-3145728
5,5,"linear_spline,linear",65536,799.587735,0.003387,3.846815e+09,10.430103,6774,0.003387,1572880,1048592,nm6d2f6a0dda092b70154a,True,False,True,-1572864
6,6,"linear_spline,linear",32768,1151.260790,0.004347,1.228171e+10,11.030974,8695,0.004347,786448,524304,nm238bd4f690b492c7d5e8,True,False,True,-786432
7,7,"linear_spline,linear",16384,1664.141578,0.004609,4.346324e+10,11.609133,9218,0.004609,393232,262160,nmc64eb7f62650bbe7dfff,True,False,True,-393216
8,8,"linear_spline,linear",8192,2413.632627,0.004276,1.666377e+11,12.167599,8551,0.004276,196624,131088,nmaac4a789ceba69493dce,True,False,True,-196608
9,9,"linear_spline,linear",4096,3499.589229,0.004722,6.688541e+11,12.716431,9443,0.004722,98320,65552,nm6502100e724c28edaaf0,True,False,True,-98304


# Final configs `wiki.csv`

,index,layers,branching factor,average error,average error %,average l2 error,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front,diff
0,0,"cubic,linear",2097152,6.843935,0.000971,8.338808e+06,3.858576,1941,0.000971,50331680,33554464,nm64b6dda5b888f4561122,True,False,True,-1000000000000
1,1,"cubic,linear",1048576,12.569999,0.007824,1.693544e+09,4.331367,15648,0.007824,25165856,16777248,nm736bee190651c4a0cd4d,True,False,True,-25165824
2,2,"cubic,linear",524288,17.346540,0.007824,1.693564e+09,4.849800,15647,0.007824,12582944,8388640,nm0ef962d126b549515ff0,True,False,True,-12582912
3,3,"cubic,linear",262144,37.360730,0.039700,4.368940e+10,5.437125,79401,0.039700,6291488,4194336,nm09179faa51abc737f7e4,True,False,True,-6291456
4,4,"cubic,linear",131072,56.880235,0.052127,7.555089e+10,6.110941,104254,0.052127,3145760,2097184,nm9d512a66510b9ac033ad,True,False,True,-3145728
5,5,"cubic,linear",65536,86.800784,0.051725,7.480006e+10,6.897752,103450,0.051725,1572896,1048608,nm5977238fee1957ddfb0c,True,False,True,-1572864
6,6,"cubic,linear",32768,144.203666,0.028193,2.277054e+10,7.838720,56387,0.028193,786464,524320,nmdf834f1da40372581102,True,False,True,-786432
7,7,"cubic,linear",16384,322.582491,0.016004,1.041720e+10,9.042062,32008,0.016004,393248,262176,nm716f02c1677fda4778b1,True,False,True,-393216
8,8,"cubic,linear",8192,935.394713,0.013853,5.112417e+10,10.570055,27707,0.013853,196640,131104,nm3dfcd5b0ee81140c1161,True,False,True,-196608
9,9,"cubic,linear",4096,3196.648491,0.013594,9.675679e+11,12.388285,27188,0.013594,98336,65568,nm2f6cb73bf7227f2a68cf,True,False,True,-98304


# Final configs `osm.csv`

...

,index,layers,branching factor,average error,average error %,average l2 error,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front,diff
0,0,"cubic,linear",2097152,29.291543,0.000118,3.421364e+05,5.720408,474,0.000118,50331680,33554464,nm64b6dda5b888f4561122,True,False,True,-1000000000000
1,1,"cubic,linear",1048576,42.057140,0.000128,1.355858e+06,6.179188,513,0.000128,25165856,16777248,nm736bee190651c4a0cd4d,True,False,True,-25165824
2,2,"cubic,linear",524288,60.239218,0.000146,5.469022e+06,6.636412,586,0.000146,12582944,8388640,nm0ef962d126b549515ff0,True,False,True,-12582912
3,3,"cubic,linear",262144,86.224495,0.000157,2.238564e+07,7.097978,629,0.000157,6291488,4194336,nm09179faa51abc737f7e4,True,False,True,-6291456
4,4,"cubic,linear",131072,122.971473,0.000171,9.142889e+07,7.563637,685,0.000171,3145760,2097184,nm9d512a66510b9ac033ad,True,False,True,-3145728
5,5,"cubic,linear",65536,175.257718,0.000243,3.743589e+08,8.036209,973,0.000243,1572896,1048608,nm5977238fee1957ddfb0c,True,False,True,-1572864
6,6,"cubic,linear",32768,249.683680,0.000292,1.531848e+09,8.515894,1169,0.000292,786464,524320,nmdf834f1da40372581102,True,False,True,-786432
7,7,"cubic,linear",16384,354.914587,0.000410,6.228043e+09,9.000914,1639,0.000410,393248,262176,nm716f02c1677fda4778b1,True,False,True,-393216
8,8,"cubic,linear",8192,498.088377,0.000499,2.435938e+10,9.481321,1998,0.000499,196640,131104,nm3dfcd5b0ee81140c1161,True,False,True,-196608
9,9,"cubic,linear",4096,682.463988,0.000595,9.074214e+10,9.939651,2380,0.000595,98336,65568,nm2f6cb73bf7227f2a68cf,True,False,True,-98304


# Final configs `books600.csv`

,index,layers,branching factor,average error,average error %,average l2 error,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front,diff
0,0,"cubic,linear",2097152,43.008532,0.000084,1.059960e+06,6.255798,507,0.000084,50331680,33554464,nm64b6dda5b888f4561122,True,False,True,-1000000000000
1,1,"cubic,linear",1048576,61.584883,0.000152,4.126926e+06,6.718883,910,0.000152,25165856,16777248,nm736bee190651c4a0cd4d,True,False,True,-25165824
2,2,"cubic,linear",524288,88.118567,0.000126,1.653084e+07,7.178909,755,0.000126,12582944,8388640,nm0ef962d126b549515ff0,True,False,True,-12582912
3,3,"cubic,linear",262144,125.940628,0.000167,6.725640e+07,7.641206,1000,0.000167,6291488,4194336,nm09179faa51abc737f7e4,True,False,True,-6291456
4,4,"cubic,linear",131072,179.474206,0.000182,2.739057e+08,8.107491,1094,0.000182,3145760,2097184,nm9d512a66510b9ac033ad,True,False,True,-3145728
5,5,"cubic,linear",65536,255.931976,0.000218,1.123779e+09,8.580681,1307,0.000218,1572896,1048608,nm5977238fee1957ddfb0c,True,False,True,-1572864
6,6,"cubic,linear",32768,364.503408,0.000323,4.589197e+09,9.061426,1937,0.000323,786464,524320,nmdf834f1da40372581102,True,False,True,-786432
7,7,"cubic,linear",16384,516.901953,0.000387,1.848883e+10,9.545637,2321,0.000387,393248,262176,nm716f02c1677fda4778b1,True,False,True,-393216
8,8,"cubic,linear",8192,725.790676,0.000449,7.283194e+10,10.023516,2695,0.000449,196640,131104,nm3dfcd5b0ee81140c1161,True,False,True,-196608
9,9,"cubic,linear",4096,1005.115865,0.000578,2.754918e+11,10.497749,3467,0.000578,98336,65568,nm2f6cb73bf7227f2a68cf,True,False,True,-98304


# Final configs `books800.csv`

,index,layers,branching factor,average error,average error %,average l2 error,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front,diff
0,0,"cubic,linear",2097152,59.359151,0.000124,2.850875e+06,6.698970,989,0.000124,50331680,33554464,nm64b6dda5b888f4561122,True,False,True,-1000000000000
1,1,"cubic,linear",1048576,84.769484,0.000130,1.107493e+07,7.163128,1042,0.000130,25165856,16777248,nm736bee190651c4a0cd4d,True,False,True,-25165824
2,2,"cubic,linear",524288,120.982300,0.000150,4.418538e+07,7.623692,1203,0.000150,12582944,8388640,nm0ef962d126b549515ff0,True,False,True,-12582912
3,3,"cubic,linear",262144,172.803398,0.000158,1.798237e+08,8.087635,1260,0.000158,6291488,4194336,nm09179faa51abc737f7e4,True,False,True,-6291456
4,4,"cubic,linear",131072,246.193558,0.000172,7.327172e+08,8.555401,1372,0.000172,3145760,2097184,nm9d512a66510b9ac033ad,True,False,True,-3145728
5,5,"cubic,linear",65536,350.691477,0.000243,2.997339e+09,9.029714,1946,0.000243,1572896,1048608,nm5977238fee1957ddfb0c,True,False,True,-1572864
6,6,"cubic,linear",32768,499.486937,0.000292,1.225911e+10,9.510875,2337,0.000292,786464,524320,nmdf834f1da40372581102,True,False,True,-786432
7,7,"cubic,linear",16384,709.896403,0.000410,4.983116e+10,9.997131,3282,0.000410,393248,262176,nm716f02c1677fda4778b1,True,False,True,-393216
8,8,"cubic,linear",8192,996.226846,0.000499,1.948866e+11,10.478534,3996,0.000499,196640,131104,nm3dfcd5b0ee81140c1161,True,False,True,-196608
9,9,"cubic,linear",4096,1364.977123,0.000596,7.259827e+11,10.937593,4771,0.000596,98336,65568,nm2f6cb73bf7227f2a68cf,True,False,True,-98304
